## from pytorch official website

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 19847959.06it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 364726.92it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6507309.15it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5021459.77it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [4]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
# All datasets are subclasses of torch.utils.data.Dataset i.e, they have __getitem__ and __len__ methods implemented. 
# Hence, they can all be passed to a torch.utils.data.DataLoader which can load multiple samples in parallel using torch.multiprocessing workers. For example:

# imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
# data_loader = torch.utils.data.DataLoader(imagenet_data,
#                                           batch_size=4,
#                                           shuffle=True,
#                                           num_workers=args.nThreads)

In [7]:
len(training_data)

60000

In [39]:
training_data.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [26]:
training_data[0][0].shape, training_data[0][0].size()

(torch.Size([1, 28, 28]), torch.Size([1, 28, 28]))

In [13]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


## Model

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
## why zero_grad()?
# In PyTorch, for every mini-batch during the training phase, 
# we typically want to explicitly set the gradients to zero before starting to do backpropagation (i.e., updating the Weights and biases) 
# because PyTorch accumulates the gradients on subsequent backward passes. 
# This accumulating behavior is convenient while training RNNs or when we want to compute the gradient of the loss summed over multiple mini-batches. 
# So, the default action has been set to accumulate (i.e. sum) the gradients on every loss.backward() call.

In [54]:
model(training_data[0][0].to(device))

tensor([[-2.8917, -4.6754, -0.7316, -3.2345, -0.9054,  2.8110, -1.1919,  2.5558,
          3.0839,  4.7300]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [48]:
model(training_data[0][0].to(device)).argmax(1)

tensor([9], device='cuda:0')

In [49]:
training_data.targets[0]

tensor(9)

In [50]:
model(training_data[0][0].to(device)).argmax(1) == training_data.targets[0]

tensor([True], device='cuda:0')

In [31]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [40]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print(f"in test(), size: {size}  ---------------------  num_batches:{num_batches} ")
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.162554  [   64/60000]
loss: 1.173938  [ 6464/60000]
loss: 1.019646  [12864/60000]
loss: 1.165064  [19264/60000]
loss: 1.064576  [25664/60000]
loss: 1.092128  [32064/60000]
loss: 1.133573  [38464/60000]
loss: 1.071291  [44864/60000]
loss: 1.132108  [51264/60000]
loss: 1.070484  [57664/60000]
in test(), size: 10000  ---------------------  num_batches:157 
Test Error: 
 Accuracy: 64.2%, Avg loss: 1.098301 

Epoch 2
-------------------------------
loss: 1.162554  [   64/60000]
loss: 1.173938  [ 6464/60000]
loss: 1.019646  [12864/60000]
loss: 1.165064  [19264/60000]
loss: 1.064576  [25664/60000]
loss: 1.092128  [32064/60000]
loss: 1.133573  [38464/60000]
loss: 1.071291  [44864/60000]
loss: 1.132108  [51264/60000]
loss: 1.070484  [57664/60000]
in test(), size: 10000  ---------------------  num_batches:157 
Test Error: 
 Accuracy: 64.2%, Avg loss: 1.098301 

Epoch 3
-------------------------------
loss: 1.162554  [   64/60000]
loss: 1.173938  [ 

## save model 
### A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [34]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## loading model


In [35]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [36]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
